## Snake vocal control game V2

Second version of the game (game 3 in main notebook), this one is partially functional.

WARNING : Due to Frozen models being used, in some program iterations the kernel can potentially crash.

Other than that the game functions normally.

In [3]:
###################################################################################################################################################################################
### I) imports
###################################################################################################################################################################################

import random
import tkinter as tk
import time
from speech_recognition import Recognizer, Microphone
import threading
from multiprocessing import Process 
import sys

###################################################################################################################################################################################
### II) Classes
###################################################################################################################################################################################

#score class : class for recording the game scores #################################################
class Score:
  """Score class : class for recording each game score. 
  Each score has :\n
     - a val (int);\n
     - a game_id (string);\n
     - a high score verrification tag (bool);\n

  methods/constructor : \n
     - Score -> the class constructor;\n
     - get_val -> getter for score.val;\n
     - get_game_id -> getter for score.game_id;\n
     - get_high_score-> getter for score.high_score;\n
     - change_val -> setter for score.val;\n
     - change_game_id -> setter for score.game_id;\n
     - change_high_score-> setter for score.high_score;"""

  def __init__(self, val=0, game_id='', high_score =False ):      
    self.val = val            #'0'     #default / val is an int
    self.game_id = game_id    #'g.0'   #default / game_id is a string : g.number (ex: 'g.2' = game 2)
    self.high_score = high_score #False #default /high score is a boolean


  def get_val(self):         #getter method that enables us to get the value of a Score object
    val= self.val
    return val

  def get_game_id(self):     #getter method that enables us to get the game_id of a Sore object
    game_id= self.game_id
    return game_id

  def get_high_score(self):         #getter method that enables us to get the verrification boolean of a Score object
    high_score_check= self.high_score
    return high_score_check

  def change_high_score(self,new_high_score):     #setter method that enables us to change the verrification boolean of a Score object
    self.high_score = new_high_score
    return self.high_score 
  
  def change_val(self,new_val):     #setter method that enables us to change the value of a Score object
    self.val = new_val
    return self.val 

  def change_game_id(self,new_id):  #setter method that enables us to change the game_id of a Sore object
    self.game_id = new_id
    return self.game_id
  
#Host class for global variables generation#################################################
class Host: 
  """Host class : class for global values generation (for different use cases in the program). 
  Each Host has :\n
     - a val (int);\n

  methods/constructor : \n
     - Host -> the class constructor;\n
     - get_val -> getter for score.val;\n
     - change -> setter for score.val;"""
  
  def __init__(self, val =''):      #constructor with '' being the default value
    self.val = val

  def get_val(self):       #getter method that enables us to get the value of a Host object
    h= self.val
    return h

  def change(self,new_val):#method to change the Host object's value
    self.val = new_val
    return self.val  
 
###################################################################################################################################################################################
### III) Class objects
###################################################################################################################################################################################

#list for game1 Score objects 
score_list =[[Score() for i in range(20) ] for j in range(3)]
hight_score_list =[Host(0) for i in range(3)]

vocal_key = Host('down')

pause_val1 = Host(True)
pause_val2 = Host(True)
pause_val3 = Host(True)

time_c = Host(1000)

#time_es = Host(4000)

size = Host(20)

G_Ind= Host(1)

god_mode= Host(-1)

color_track = Host(0)

#lists with objects#################################################
random_snake_list = [Host(4) for i in range(5)]

#Host objects for keeping track of score evolution
S_list= [[Host(0),Host(0), hight_score_list[j]] for j in range(3)] # each line : #1 is score_count / #2 is game_count / #3 is High_score

#objects for snake segments coordinates (x,y)
circ_coord_list = [[Host(),Host()] for i in range(1,(size.get_val()+1)) ]

circ_coord_list2 = [[[Host(),Host()] for i in range(1,(size.get_val()+1))] for j in range(5)]

#others
word_list=['left', 'right', 'up', 'down', 'goodbye']

###################################################################################################################################################################################
### IV) root creation
###################################################################################################################################################################################

#################################################
### Root and canvas creation
#################################################
rootv =  tk.Tk()
rootv.geometry("650x650")
rootv.title("Classic snake")
rootv.configure(background='black')

    #canvas creation
GCanvas = tk.Canvas(rootv, width=500, height=500, bg='lightblue')
GCanvas.pack()
GCanvas.grid(row =3, column=3)

n = size.get_val()

###################################################################################################################################################################################
### IV) function : game
###################################################################################################################################################################################

def game():
#function for vocal python game creation #################################################

#################################################
### Functions
#################################################

#functions for moving the snake#################################################
            
    def move_snake2(circ): #second function for moving the snake (part1): for each snake segments the coordinates of the previous segment are collected.
                                #After the previous segment is moved the curent segment is moved using the previous segments previous coordinates.
                                #here in this function only the first snake segment is moved depending on th keyboard input.


        for i in range(n):
            circ_coord_list[i][0].change(GCanvas.coords(circle_list[(n-1)-i])[0]+10)
            circ_coord_list[i][1].change(GCanvas.coords(circle_list[(n-1)-i])[1]+10)

        if 'up' in vocal_key.get_val():
            GCanvas.move(circ, 0, -10)
            move_snake3()
        if 'down' in vocal_key.get_val():
            GCanvas.move(circ, 0, 10)
            move_snake3()
        if 'left' in vocal_key.get_val():
            GCanvas.move(circ, -10, 0)
            move_snake3()
        if 'right' in vocal_key.get_val():
            GCanvas.move(circ, 10, 0)
            move_snake3()

    def move_snake3(): #second function for moving the snake (part2) : movement update for each segment except the first one (moved in first part)

        for i in range(1,n):
            GCanvas.move(circle_list[(n-1)-i], circ_coord_list[i-1][0].get_val()-circ_coord_list[i][0].get_val(),circ_coord_list[i-1][1].get_val()-circ_coord_list[i][1].get_val())

#functions for snake environment interaction#################################################

    def eat_apple(): #function for eating an apple. When the snakes first degment (circ) touches the apple (circ coordinates in apple)
                    #the next snake segment is coloured and the apple is moved randomply within the canvas
        x_ap = GCanvas.coords(apple)[0]
        y_ap = GCanvas.coords(apple)[1]
        interv1 = 28

        if ((x_ap-interv1)<(GCanvas.coords(circ)[0])<(x_ap+interv1) and (y_ap-interv1)<(GCanvas.coords(circ)[1])<(y_ap+interv1)):
            
            GCanvas.move(apple, random.randrange(10-x_ap,int(GCanvas.winfo_reqwidth())-x_ap-10),random.randrange(10-y_ap,int(GCanvas.winfo_reqheight())-y_ap-10))
            S_list[2][0].change(S_list[2][0].get_val()+1)

            for i in range(n):
                if ( GCanvas.itemconfig(circle_list[(n-1)-i])['fill'][4]=='lightblue'):
                    GCanvas.itemconfig(circle_list[(n-1)-i], fill='blue',outline="blue")
                    break

    def hit_wall(rootv):
        x_min_lim =5
        x_max_lim =GCanvas.winfo_reqwidth()-15
        y_min_lim =5
        y_max_lim =GCanvas.winfo_reqheight()-15

        interv2 = 6

        if (x_max_lim<(GCanvas.coords(circ)[0]) or (GCanvas.coords(circ)[0])<x_min_lim or y_max_lim<(GCanvas.coords(circ)[1]) or (GCanvas.coords(circ)[1])<y_min_lim ): 
            #checks if a wall has been hit 
            G_Ind.change(2)
            #game_over(Game_over)
            rootv.destroy()

        else :
            for i in range(n-3):
                x_circ = GCanvas.coords(circle_list[i])[0]
                y_circ = GCanvas.coords(circle_list[i])[1]

                if ((x_circ-interv2)<(GCanvas.coords(circle_list[n-1])[0])<(x_circ+interv2) and (y_circ-interv2)<(GCanvas.coords(circle_list[n-1])[1])<(y_circ+interv2)):

                    if( GCanvas.itemcget(circle_list[i], "fill")=='blue'):
                        G_Ind.change(2)
                        #game_over(Game_over)
                        rootv.destroy()
                        break

#function for pausing the game#################################################              
    def pause():
        if pause_val3.get_val()==True:
            pause_val3.change(False)

        else:
            pause_val3.change(True)
            
#functions for time update#################################################

    def clock1(): #function to enable game update

       # if pause_val3.get_val()==True:
            
            #threading.Thread(target=move_snake2).start()
            #move_snake1()
            move_snake2(circ)
            eat_apple()
            hit_wall( rootv)

            if (S_list[2][2].get_val()<S_list[2][0].get_val()):
                S_list[2][2].change(S_list[2][0].get_val())

            
            if 'goodbye' in vocal_key.get_val():
                rootv.destroy()

            score_label.config(text = "Score : "+str(S_list[2][0].get_val()) +"  High score : " +str(S_list[2][2].get_val()))

            rootv.after(500, clock1) # updates every 500 ms

    def clock2():
        #if pause_val3.get_val()==True:
        #threading.Thread(target=move_snake1).start()
        move_snake1()
        rootv.after(400, clock2) 


#function shape creation#################################################

    def create_circle2(x ,y ,r ,color):  #function to create circles using create_oval (by modifying the oval's bonding box)
                                        #arguments are the x and y coordinates of the circle's center, r it's radious and the desired color
                                        
        #circle's bounding box cordinates
        ul_x = x-r     # uper left x coordinate
        ul_y = y-r     # uper left y coordinate
        lr_x = x+r     # lower right x coordinate
        lr_y = y+r     # lower right y coordinate

        return GCanvas.create_oval(ul_x,ul_y,lr_x,lr_y, fill= color, outline=color)

#################################################
### Shape creation
#################################################
    
    #snake 
    circle_list = [create_circle2(200-10*(n-i), 60, 10, 'lightblue') for i in range((n-1))]
    circ=create_circle2(200, 60, 10, 'green')
    circle_list.append(circ)

    #apple
    apple=create_circle2(200, 200, 20, 'red')

#################################################
### Labels and function execution 
#################################################

    #labels
    score_label = tk.Label(rootv)
    score_label.grid(row=13, columnspan=20)


    for i in range(1):
        tk.Label(rootv,text = ' ',bg='black').grid(row=i, column=0)
        #tk.Label(rootg,text = ' ').grid(row=0, column=i)
        

    sp_label3 = tk.Label(rootv,text = ' ',bg='black')     #label for displaying the scale (distance)
    sp_label3.grid(row=12, column=0)


    #clock update (for distance and gravitational force update)

    clock1()
    #clock2()

    #buttons

    exit_button = tk.Button(rootv, text="Rage quit", command=rootv.destroy)  #button to kill GUI window
    exit_button.grid(row=13, column=1)

    #Pause_button = tk.Button(rootv, text="⏯", command=pause)  #button to kill GUI window
    #Pause_button.grid(row=13, column=2)

###################################################################################################################################################################################
### IV) function : voc
################################################################################################################################################################################### 

#function for vocal word recognition #################################################
def voc(): 
    micro = Microphone(device_index=1) #1
    while 1==1 :         
        with micro as source:
                            recognizer = Recognizer()
                            recognizer.adjust_for_ambient_noise(source,duration=0.5)
                            #print("vocalise command")
                            #print('')
                            recorded_audio = recognizer.listen(source)
        try:
                            text = recognizer.recognize_google( recorded_audio, language="en-EN")
                            #text = recognizer.recognize_sphinx( recorded_audio, language="en-EN")
                            for word in word_list:
                                if word in text : 
                                    vocal_key.change(text)
                                    print(text)
                                    #print('')
                                    break

        except Exception as ex:
                                text = "Fail"
                                print('Failed word recognition')
                                #print('')

###################################################################################################################################################################################
### IV) threading + multiprocessing
###################################################################################################################################################################################
            
#p1 = Process(target=voc)
p2 = Process(target=game())
#p1.start()
p2.start()   
#p1.join()   
                  
#voc()
threading.Thread(target=voc).start()

#game()
rootv.mainloop()




go left


Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
go left to work on Monday Tuesday Saturday
Failed word recognition
Failed word recognition
go left to work together hey Google
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed word recognition
Failed wo